# Customer Lifetime Value or CLV (Total profit derived from customer)
## Setting (Online Retail)- 
    1. Non-Contractual ("death" of customer unobserved) 
    2. Continuous Purchases ("non-periodicity" of purchases - can happen anytime)

### Importing libraries

**Required lib(s)**

In [13]:
import pandas as pd  #Data manipulations
import datetime as dt  #Date-time manipulations

**Good-to-have**

In [68]:
import warnings
warnings.filterwarnings('ignore')

### Importing data and high-level inspection

**Imports**

In [43]:
#Importing file from the web into a Pandas DF
df=pd.read_excel("http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx")

**High-level Inspection**

In [94]:
#Checking a few rows from the top
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [98]:
#Getting info (rows, cols, datatypes)
print(df.info(),'\n')

#Getting some Summary Stats (including all cols types)
df.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None 



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909.0,541909,540455,541909.000000,541909,541909.000000,406829.000000,541909
unique,25900.0,4070,4223,NaN,23260,NaN,NaN,38
top,573585.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-10-31 14:41:00,NaN,NaN,United Kingdom
freq,1114.0,2313,2369,NaN,1114,NaN,NaN,495478
first,NaN,NaN,NaN,NaN,2010-12-01 08:26:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,9.552250,NaN,4.611114,15287.690570,NaN
std,NaN,NaN,NaN,218.081158,NaN,96.759853,1713.600303,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,NaN,1.250000,13953.000000,NaN


### Data Cleaning and Manipulations

**Cleaning - Character columns**

In [71]:
#Filtering - Removing rows where customer ID is NULL
df1=df[df['CustomerID'].notnull()]

**Cleaning - Numeric columns**

In [72]:
#Filtering - Removing rows where quantity and unit-price -ve => "returns"
##Note: Rows where unit price <=0 is (40); but for unit price <0 it's (zero)
##Note: Rows where quantity <=0 is (8905); and it's for quantity <0 
df2=df1[(df1['Quantity']>0)] 

**Cleaning - Datetime columns**

In [73]:
#Extracting dates out of datetime column
df2['InvoiceDate']=df2['InvoiceDate'].dt.date

**Manipulations**

In [101]:
#Adding a required column for our model (i.e. Revenue)
df2['Revenue']=df2['UnitPrice']*df2['Quantity']

#Keeping only those columns that we need for analysis
df3=df2[['CustomerID','Quantity','Revenue']]

#High-level inspection of our final analytical data
print(df3.info(), '\n')
print(df3.describe())
df3.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 3 columns):
CustomerID    397924 non-null float64
Quantity      397924 non-null int64
Revenue       397924 non-null float64
dtypes: float64(2), int64(1)
memory usage: 22.1 MB
None 

          CustomerID       Quantity        Revenue
count  397924.000000  397924.000000  397924.000000
mean    15294.315171      13.021823      22.394749
std      1713.169877     180.420210     309.055588
min     12346.000000       1.000000       0.000000
25%     13969.000000       2.000000       4.680000
50%     15159.000000       6.000000      11.800000
75%     16795.000000      12.000000      19.800000
max     18287.000000   80995.000000  168469.600000


,CustomerID,Quantity,Revenue
0,17850.0,6,15.30
1,17850.0,6,20.34
2,17850.0,8,22.00
3,17850.0,6,20.34
4,17850.0,6,20.34
